In [4]:
import asyncio
import aiohttp
import pandas as pd

# Парсер для получения данных с westmetall
async def parser(metall_str: str):
    url = f'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall_str}_cash'

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.text()

    df = pd.read_html(data)[0]
    cleared_df = df[df['date'] != 'date']
    cleared_df['date'] = pd.to_datetime(cleared_df['date'])
    cleared_df['LME Aluminium Cash-Settlement'] = pd.to_numeric(cleared_df['LME Aluminium Cash-Settlement'])
    cleared_df['LME Aluminium 3-month'] = pd.to_numeric(cleared_df['LME Aluminium 3-month'])
    cleared_df['LME Aluminium stock'] = pd.to_numeric(cleared_df['LME Aluminium stock']) / 1000
    
    return cleared_df

# Список источников, нам понадобится информация по 5 видам сырья:
# алюминий, медь, свинец, никель и цинк
metalls = ['Al', 'Cu', 'Pb', 'Ni', 'Zn']

async def main():
    tasks = []
    async with aiohttp.ClientSession() as session:
        for metall in metalls:
            task = asyncio.ensure_future(parser(metall))
            tasks.append(task)
        results = await asyncio.gather(*tasks)

    al_df = results[0]
    cu_df = results[1]
    pb_df = results[2]
    ni_df = results[3]
    zn_df = results[4]

    # Дальнейшая обработка данных

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


RuntimeError: This event loop is already running